Prueba de progreso 2

Imports

In [ ]:
import os
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import seaborn as sns
import glob

Carga de datos

In [ ]:
def cargar_datos(ruta_base):
    datos_completos = []
    for persona_id in os.listdir(ruta_base):
        carpeta_persona = os.path.join(ruta_base, persona_id)
        if not os.path.isdir(carpeta_persona):
            continue
        for archivo in glob.glob(os.path.join(carpeta_persona, '*.csv')):
            df = pd.read_csv(archivo)
            df['persona_id'] = persona_id
            df['ruta_id'] = os.path.basename(archivo).split('.')[0]
            datos_completos.append(df)
    return pd.concat(datos_completos, ignore_index=True)

Preprocesamiento

#Eliminar la correlación general con la pelvis

In [ ]:
def restar_pelvis(df):
    columnas = [col for col in df.columns if any(eje in col for eje in ['X', 'Y', 'Z']) and 'pelvis' not in col]
    for eje in ['X', 'Y', 'Z']:
        pelvis_col = f'pelvis_{eje}'
        for col in columnas:
            if col.endswith(eje):
                df[col] = df[col] - df[pelvis_col]
    return df

#Ventaneo

In [ ]:
def segmentar_ventanas(df, tam_ventana=240):
    ventanas = []
    for (persona, ruta), grupo in df.groupby(['persona_id', 'ruta_id']):
        n = len(grupo)
        for i in range(0, n - tam_ventana + 1, tam_ventana):
            ventana = grupo.iloc[i:i+tam_ventana].copy()
            ventana['ventana_id'] = f'{persona}_{ruta}_{i//tam_ventana}'
            ventanas.append(ventana)
    return ventanas

#Normalización por persona y por ventana

In [ ]:
def normalizar_ventana(ventana):
    columnas_xyz = [col for col in ventana.columns if any(col.endswith(eje) for eje in ['X', 'Y', 'Z'])]
    scaler = StandardScaler()
    ventana[columns_xyz] = scaler.fit_transform(ventana[columns_xyz])
    return ventana

Extracción de Características

In [ ]:
def extraer_caracteristicas(ventana):
    features = {}
    articulaciones = set(col[:-2] for col in ventana.columns if '_' in col and col[:-2] != 'pelvis')
    for art in articulaciones:
        for eje in ['X', 'Y', 'Z']:
            col = f'{art}_{eje}'
            señal = ventana[col].values
            features[f'{art}_{eje}_media'] = np.mean(señal)
            features[f'{art}_{eje}_std'] = np.std(señal)
            features[f'{art}_{eje}_min'] = np.min(señal)
            features[f'{art}_{eje}_max'] = np.max(señal)
            features[f'{art}_{eje}_energia'] = np.sum(señal**2)
    features['ventana_id'] = ventana['ventana_id'].iloc[0]
    return features

Pipeline General

In [ ]:
def procesar_todo(ruta_base):
    df = cargar_datos(ruta_base)
    df = restar_pelvis(df)
    ventanas = segmentar_ventanas(df)
    ventanas = [normalizar_ventana(v) for v in ventanas]
    caracteristicas = [extraer_caracteristicas(v) for v in ventanas]
    return pd.DataFrame(caracteristicas)

Clustering y Visualización

In [ ]:
def clustering_y_visualizacion(df_features, n_clusters=5):
    X = df_features.drop(columns=['ventana_id'])
    pca = PCA(n_components=2)
    X_pca = pca.fit_transform(X)
    
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    labels = kmeans.fit_predict(X)
    
    plt.figure(figsize=(10, 6))
    sns.scatterplot(x=X_pca[:,0], y=X_pca[:,1], hue=labels, palette='Set2')
    plt.title("Clustering de Ventanas de Movimiento Humano (PCA)")
    plt.xlabel("Componente Principal 1")
    plt.ylabel("Componente Principal 2")
    plt.legend(title="Cluster")
    plt.show()

    df_features['cluster'] = labels
    return df_features